# Kütüphane ve Veri Seti Import Etme 
**Gerekli kütüphaneleri import ediyoruz.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

**Veri setimizi okuyup dataset adlı değişkene atıyoruz.**

In [ ]:
dataset=pd.read_csv('../input/tabletpc-priceclassification/tablet.csv')

# Veriyi Tanıma ve Sindirme Süreci
**Veri setimizi dataFrame'e çeviriyoruz.**

In [ ]:
df = pd.DataFrame(dataset)

Veri setimizin ilk 10 gözlemini listeliyoruz.

In [ ]:
df.head(10)

**Veri çerçevemizdeki gözlem ve değişken sayılarını yazdırıyoruz.**

In [ ]:
df.shape

**Değikenlerimizin tiplerini yazdırıyoruz.**

In [ ]:
df.dtypes

**Veri setimizle alakalı genel inceleme yapıyoruz.**

In [ ]:
df.info()

**Veri setimizdeki sayısal değişknlerin istatistiki bilgilere genel olarak bakıyoruz.**

In [ ]:
df.describe().T

**RAM değişkenimizin standart sapmasının yüksek olması dikkatimizi çekiyor.

**Sayısal değikenler arasındaki korelasyon ilişkisini görmek için Korelasyon Matrisi çizdiriyoruz.**

In [ ]:
corr = df.corr()
corr

Değişken sayımız fazla olduğu için korelasyon tablosunu okumak biraz zor. Bunun için ısı haritasi çizdirelim.

In [ ]:
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values);

**ArkaKameraMp ile OnKameraMp değişkenlerimiz arasında bir ilişki olduğunu görüyoruz. Ayrıca CozunurlukGenislik ve CozunurlukGenislik arasında da çok zayıfta olsa bir ilişki olduğunu söyleyebiliriz.**

# Değişkenlerin Dağılımı

Hedef değişkenimizin dengeli dağılıp dağılmadığına bakalım.

In [ ]:
sns.countplot(df["FiyatAraligi"]);

Countplot grafiğinden dağılımın dengeli olduğunu görüyoruz ama sayısal olarakta dağılıma bir bakalım.

In [ ]:
df["FiyatAraligi"].value_counts()

Hedef değişkenimizin tamamen dengeli dağıldığını söyleyebiliriz.

Veri setimizideki değişkenlerimizi numeric ve categorical olanları seçip ayrı değişkenlere atadık.

In [ ]:
num_df = df.select_dtypes(include = ['float64', 'int64'])
cat_df = df.select_dtypes(include = ['object'])

Numeric değişkenlerimizi histogram grafiği ile görselleştirelim.

In [ ]:
num_df.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

Kategorik değişkenlerimizi görselleştirelim.

In [ ]:
sns.countplot(df["Bluetooth"]);

In [ ]:
sns.countplot(df["CiftHat"]);

In [ ]:
sns.countplot(df["4G"]);

In [ ]:
sns.countplot(df["3G"]);

In [ ]:
sns.countplot(df["Dokunmatik"]);

In [ ]:
sns.countplot(df["WiFi"]);

In [ ]:
sns.countplot(df["Renk"]);

Veri setimizdeki kategorik değişkenlerimizden 3G hariç diğerlerinin dağılımının dengeli olduğunu söyleyebiliriz.

# Veri Ön İşleme

**Veri çerçevemizde eksik veri var mı onu kontrol ediyoruz.**

In [ ]:
df.isna().sum()

**Veri çerçevemizde OnKameraMP ve RAM deişkenlerinde eksik veriler olduğunu tespit ettik.

Eksik verilerimizin olduğu gözlemleri inceleyelim.

In [ ]:
df[df["RAM"].isnull()]

Ram değişkenimizin eksik olduğu gözlemlere baktığımızda hepsinin fiyat aralığının 'Pahalı' olduğunu gözlemlemekteyiz. Burdaki eksik verileri fiyat aralığı pahalı olan gözlemlerin ortamalasını alarak doldurabiliriz.

In [ ]:
df.groupby("FiyatAraligi")[["RAM"]].mean()

Fiyatı pahalı olan modellerin RAM'lerinin ortalamasının 3449 olduğu gözlemledik. Eksik verileri bu değer ile dolduralım.

In [ ]:
df['RAM'] = df['RAM'].fillna(value=3449)

RAM değişkenimizde eksik veri kaldı mı kontrol edelim.

In [ ]:
df.isna().sum()["RAM"]

OnKameraMp değişkenimizdeki boş değerleri listeleyelim.

In [ ]:
df[df["OnKameraMP"].isnull()]

Burada da eksik özniteliğe sahip modellerin FiyatAraligi değişkenlerinin Çok Ucuz olduğunu gözlemledik. Eksik verilerimizi fiyatı çok ucuz olan modellerin ortalamalarına göre doldurlarım.

In [ ]:
df.groupby("FiyatAraligi")[["OnKameraMP"]].mean()

Çok ucuz olan modellerin OnKameraMp değişkeninin ortalama değerinin yaklaşık olarak 4 olduğunu gözlemledik. Eksik verilerimizi bu değere göre dolduralım.

In [ ]:
df['OnKameraMP'] = df['OnKameraMP'].fillna(value=4)

Doldurma işleminin başarılı bir şekilde yapılıp yapılmadığını kontrol edelim.

In [ ]:
df.isna().sum()["OnKameraMP"]

Tüm veri setimizde eksik veri kaldımı tekrar bakalım.

In [ ]:
df.isna().sum()

# Kategorik Değerleri Dönüştürme

Kategorik verileri dönüştürmeden önce benzersiz değerlerine bakalım.

In [ ]:
df["Bluetooth"].unique()

In [ ]:
df["CiftHat"].unique()

In [ ]:
df["4G"].unique()

In [ ]:
df["3G"].unique()

In [ ]:
df["Dokunmatik"].unique()

In [ ]:
df["WiFi"].unique()

In [ ]:
df["Renk"].unique()

In [ ]:
df.groupby("Renk")[["FiyatAraligi"]]

Renk hariç diğer kategorik değişkenlerimizin gözlemleri 'Var' ve 'Yok' olarak tanımlanmış. Bunları doğrudan '1' ve '0' olarak değiştirebiliriz. Ayrıca değişikliğin DataFrame'e uygulanabilmesi için metodumuzun default olarak gelen inplace özelliğini true olarak değiştiriyoruz.

In [ ]:
df.replace("Var",1,inplace = True)
df.replace("Yok",0,inplace = True)

**Renk değişkenimizin FiyatAraligi'na etkisi olup olmadığını belirlemek için Pandas kütüphanesi içerisinde gelen crosstab metodunu kullanarak iki kategorik değişkenimizi karşılaştırıyoruz.**

In [ ]:
renk_fiyat_crosstab = pd.crosstab(index=df["Renk"], columns=df["FiyatAraligi"])
renk_fiyat_crosstab

**Görüntülediğimiz tabloda Renk değişkenimizin FiyatAraligi'na pek etkisi olmadığını anlamaktayız. O sebeple Renk değişkenimizi DataFrame'den silebiliriz.**

In [ ]:
df.drop(['Renk'], axis=1, inplace=True)

In [ ]:
df.head()

# Eğitim ve Test Veri Setleri Oluşturma

**Bağımlı ve bağımsız değişkenlerimizi x ve y olarak ikiye bölelim.**

In [ ]:
y = df["FiyatAraligi"]
x = df.drop(["FiyatAraligi"], axis=1)

**x ve y'yi kontrol edelim.**

In [ ]:
y.head()

In [ ]:
x.head()

Yeni veri setimizi %25, %75 olarak eğitim ve test verisi olarak ikiye bölelim.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=42)

In [ ]:
x_train

In [ ]:
y_test

# GaussianNB Modeli



GaussianNB modeli yaratıp, eğitimini gerçekleştiriyoruz.

In [ ]:
nb=GaussianNB()
nb.fit(x_train, y_train)

x_test gözlemlerinin tahmin sonucunu y_pred değişkenimize atıyoruz. 

In [ ]:
y_pred = nb.predict(x_test)

Doğruluk skorumuzu hesaplayalım.

In [ ]:
accuracy_score(y_test, y_pred)

Oluşturduğumuz model ile %81 doğruluk skoru elde etmiş olduk.


Karmaşıklık matrisimizi oluşturalım

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
sns.heatmap(karmasiklik_matrisi,cbar=False,annot=True,cmap="Greens",fmt="d")

# Desicion Tree Modeli

Karar ağacı modelimizi oluşturalım.

In [ ]:
tree_model = DecisionTreeClassifier()

Modelimizi veri setimiz ile eğitelim.

In [ ]:
tree_model.fit(x_train,y_train)

Modelimizin doğruluk skoru yazdıralım.

In [ ]:
tree_model.score(x_test, y_test)

Oluşturduğumuz model %83 başarılı olduğunu görmüş olduk.

Varsayılan parametresini gini'den entropy'e çevirerek modelimizi yeniden eğitelim.

In [ ]:
tree_model_entropy = DecisionTreeClassifier(criterion = 'entropy')

In [ ]:
tree_model_entropy.fit(x_train, y_train)

Yeni oluşturduğumuz modelin doğruluk skorunu hesaplayalım.

In [ ]:
tree_model_entropy.score(x_test, y_test)

Varsayılan parametremiz 'gini' olarak kullandığımızda %83 başarı elde etmiştik. Entropy ile başarı oranımızı %87 olarak güncellemiş olduk. Karar ağacı modelimizi entropy ile kullanmamız daha doğru olacaktır.

# KNN Modeli

KNN modelimizi oluşturalım.

In [ ]:
knn = KNeighborsClassifier()

Modelimizi eğitelim.

In [ ]:
knn.fit(x_train, y_train)

In [ ]:
y_pred = knn.predict(x_test)

Doğruluk skorumuzu görüntüleyelim.

In [ ]:
accuracy_score(y_test, y_pred)

Modelimizi parametre optimizasyonu yaparak tekrar eğitip sonuçlarına bakalım.

In [ ]:
knn_params = {"n_neighbors": np.arange(2,15)}

In [ ]:
knn_new = GridSearchCV(knn, knn_params, cv = 3)
knn_new.fit(x_train, y_train)

Modelimizin için en iyi parametre ve skoru belirleyelim.

In [ ]:
print("En iyi skor: " + str(knn_new.best_score_))
print("En iyi parametreler: " + str(knn_new.best_params_))

En iyi parametrenin 9 olduğunu öğrenmiş olduk. Modelimizi buna göre revize edelim.

In [ ]:
knn = KNeighborsClassifier(9)
knn_tuned = knn.fit(x_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(x_test)

Modelimizin doğruluk skorunu hesaplayalım.

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
Modelimizin doğruluk skorunu optimizasyon sonrası %93'e yükseltmiş olduk. 